In [12]:
import os
import glob
import json
from decimal import Decimal
from pathlib import Path

import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.spatial.transform import Rotation, Rotation as R
from tqdm import tqdm  # progress bar
# If tqdm is missing, install once with:  !pip install tqdm


In [13]:
# === CONFIG: YOUR PATHS (using Path as you prefer) ===

# Folder with target boxes (e.g. SOUTH LiDAR labels)
input_folder_path_target_boxes = Path(r"C:\Users\mcsmu\Desktop\intersection\labels_point_clouds\s110_lidar_ouster_north")

# Folder with source boxes (e.g. NORTH LiDAR labels)
input_folder_path_source_boxes = Path(r"C:\Users\mcsmu\Desktop\intersection\labels_point_clouds\s110_lidar_ouster_south")

# Output folder for fused labels
output_folder_path_fused_boxes = Path(r"C:\Users\mcsmu\Desktop\mm3\Fused json")

# If True -> assume both in same coordinate system (no transformation)
# If False -> apply transformation matrix like in the original script
same_origin = False

# Matching threshold (in meters, center-to-center in x,y)
distance_threshold = 3.0

# ===========================================

# Create output folder if it doesn't exist
os.makedirs(output_folder_path_fused_boxes, exist_ok=True)


def sorted_files(folder, pattern):
    """
    Returns a sorted list of files (full paths as strings) matching pattern inside folder.
    """
    folder = Path(folder)
    if not folder.exists():
        return []
    return sorted(glob.glob(str(folder / pattern)))


input_files_target = sorted_files(input_folder_path_target_boxes, "*.json")
input_files_source = sorted_files(input_folder_path_source_boxes, "*.json")

print("Number of target JSONs:", len(input_files_target))
print("Number of source JSONs:", len(input_files_source))


Number of target JSONs: 2160
Number of source JSONs: 2160


In [14]:
class DecimalEncoder(json.JSONEncoder):
    """
    Encoder to make sure Decimal objects are converted to strings in JSON,
    same behavior as in the original script.
    """
    def default(self, obj):
        if isinstance(obj, Decimal):
            return str(obj)
        return json.JSONEncoder.default(self, obj)


def store_boxes_in_open_label(json_data, output_folder_path, output_file_name):
    """
    Save json_data into output_folder_path/output_file_name using DecimalEncoder.
    output_folder_path is a Path object.
    """
    out_path = output_folder_path / output_file_name  # Path join
    with out_path.open("w", encoding="utf-8") as f:
        json_string = json.dumps(json_data, ensure_ascii=True, indent=4, cls=DecimalEncoder)
        f.write(json_string)


def get_attribute_by_name(attr_list, name):
    """
    Replacement for VisualizationUtils.get_attribute_by_name from the project.
    attr_list is a list of dicts like: [{"name": "score", "val": 0.9}, ...]
    """
    if attr_list is None:
        return None
    for a in attr_list:
        if a.get("name") == name:
            return a
    return None


def center_to_center_dist(position_3d_target, position_3d_source):
    """
    Distance between centers, only in x,y (same as original code).
    """
    return np.sqrt(
        (position_3d_source[0] - position_3d_target[0]) ** 2
        + (position_3d_source[1] - position_3d_target[1]) ** 2
    )


In [15]:
# Choose the correct transformation matrix depending on folder names,
# exactly like in the original script, but now with Path objects.

target_str = str(input_folder_path_target_boxes)
source_str = str(input_folder_path_source_boxes)

if (
    "s110_lidar_ouster_south" in target_str
    and "s110_lidar_ouster_north" in source_str
):
    # transformation matrix from lidar north to lidar south (inverse of given)
    transformation_matrix = np.linalg.inv(np.array(
        [
            [9.58895265e-01, -2.83760227e-01, -6.58645965e-05,  1.41849928e00],
            [2.83753514e-01,  9.58874128e-01, -6.65957109e-03, -1.37385689e01],
            [1.95287726e-03,  6.36714187e-03,  9.99977822e-01,  3.87637894e-01],
            [0.00000000e00,   0.00000000e00,  0.00000000e00,   1.00000000e00],
        ],
        dtype=float,
    ))
elif (
    "s110_lidar_ouster_north" in target_str
    and "s110_lidar_ouster_south" in source_str
):
    # transformation from lidar south to lidar north
    transformation_matrix = np.array(
        [
            [9.58895265e-01, -2.83760227e-01, -6.58645965e-05,  1.41849928e00],
            [2.83753514e-01,  9.58874128e-01, -6.65957109e-03, -1.37385689e01],
            [1.95287726e-03,  6.36714187e-03,  9.99977822e-01,  3.87637894e-01],
            [0.00000000e00,   0.00000000e00,  0.00000000e00,   1.00000000e00],
        ],
        dtype=float,
    )
else:
    # Fallback: identity (no transform) if folder names don't match expected patterns.
    print("Warning: folder names not recognised for north/south; using identity matrix.")
    transformation_matrix = np.eye(4, dtype=float)

transformation_matrix


array([[ 9.58895265e-01, -2.83760227e-01, -6.58645965e-05,
         1.41849928e+00],
       [ 2.83753514e-01,  9.58874128e-01, -6.65957109e-03,
        -1.37385689e+01],
       [ 1.95287726e-03,  6.36714187e-03,  9.99977822e-01,
         3.87637894e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [16]:
def assignment(objects_target, objects_source, distance_threshold):
    """
    Perform matching between target and source 3D objects using Hungarian algorithm
    on center distance (x,y) after transforming source into target frame if needed.
    Returns:
        unmatched_target_indices,
        unmatched_source_indices,
        matches (list of 1x2 arrays with [target_idx, source_idx])
    """
    iou_dst = np.zeros((len(objects_target), len(objects_source)))

    for id_s, object_target in enumerate(objects_target):
        position_3d_target = np.array(object_target["object_data"]["cuboid"]["val"])[:3]

        for id_n, object_source in enumerate(objects_source):
            position_3d_source = np.array(object_source["object_data"]["cuboid"]["val"])[:3]

            if not same_origin:
                position_3d_source = np.matmul(
                    transformation_matrix,
                    np.array([position_3d_source[0], position_3d_source[1], position_3d_source[2], 1.0]),
                )

            distance = center_to_center_dist(position_3d_target[:2], position_3d_source[:2])

            # if distance too large -> huge cost, effectively "no match"
            if distance > distance_threshold:
                distance = 999999.0

            iou_dst[id_s, id_n] = distance

    # Hungarian algorithm: minimize total distance
    matched_idx_target, matched_index_source = linear_sum_assignment(iou_dst)
    matched_idx = np.column_stack((matched_idx_target, matched_index_source))

    unmatched_target, unmatched_source = [], []

    for id_s, _ in enumerate(objects_target):
        if id_s not in matched_idx[:, 0]:
            unmatched_target.append(id_s)

    for id_n, _ in enumerate(objects_source):
        if id_n not in matched_idx[:, 1]:
            unmatched_source.append(id_n)

    matches = []
    for idx in matched_idx:
        if iou_dst[idx[0], idx[1]] > distance_threshold:
            unmatched_target.append(idx[0])
            unmatched_source.append(idx[1])
        else:
            matches.append(idx.reshape(1, 2))

    return unmatched_target, unmatched_source, matches


In [17]:
occlusion_name_to_level_mapping = {
    "NOT_OCCLUDED": 0,
    "PARTIALLY_OCCLUDED": 1,
    "MOSTLY_OCCLUDED": 2,
    "": 3,
}

occlusion_level_to_name_mapping = {
    0: "NOT_OCCLUDED",
    1: "PARTIALLY_OCCLUDED",
    2: "MOSTLY_OCCLUDED",
    3: "",
}

frame_id = 0
num_matches = 0

# tqdm over paired files
for path_target, path_source in tqdm(
    list(zip(input_files_target, input_files_source)),
    total=len(input_files_target),
    desc="Fusing frames"
):
    boxes_file_name_target = os.path.basename(path_target)
    boxes_file_name_source = os.path.basename(path_source)

    # Load JSONs
    with open(path_target, "r", encoding="utf-8") as f_t:
        json_label_target = json.load(f_t)
    with open(path_source, "r", encoding="utf-8") as f_s:
        json_label_source = json.load(f_s)

    # Collect all 3D objects from target (e.g. south)
    objects_3d_target_list = []
    for frame_idx, frame_obj in json_label_target["openlabel"]["frames"].items():
        for obj_id, objects_3d_target in frame_obj["objects"].items():
            objects_3d_target_list.append(objects_3d_target)

    # Collect all 3D objects + IDs from source (e.g. north)
    objects_3d_source_list = []
    objects_3d_source_ids_hex = []
    for frame_idx, frame_obj in json_label_source["openlabel"]["frames"].items():
        for obj_id, objects_3d_source in frame_obj["objects"].items():
            objects_3d_source_list.append(objects_3d_source)
            objects_3d_source_ids_hex.append(obj_id)

    # Matching
    unmatched_target_ids, unmatched_source_ids, matches_ids = assignment(
        objects_3d_target_list,
        objects_3d_source_list,
        distance_threshold=distance_threshold,
    )

    if len(objects_3d_source_list) > len(objects_3d_target_list):
        print(f"[WARN] Frame {frame_id}: more source objects than target objects.")

    # --- Fuse matched pairs ---
    for matches in matches_ids:
        match_id_target = matches[0, 0]
        match_id_source = matches[0, 1]

        object_3d_target = objects_3d_target_list[match_id_target]
        object_3d_source = objects_3d_source_list[match_id_source]
        num_matches += 1

        # Positions
        position_3d_target = np.array(object_3d_target["object_data"]["cuboid"]["val"])[:3]
        position_3d_source = np.array(object_3d_source["object_data"]["cuboid"]["val"])[:3]

        if not same_origin:
            position_3d_source = np.matmul(
                transformation_matrix,
                np.array([position_3d_source[0], position_3d_source[1], position_3d_source[2], 1.0]),
            )

        position_3d_merged = (
            np.mean([position_3d_target[0], position_3d_source[0]]),
            np.mean([position_3d_target[1], position_3d_source[1]]),
            np.mean([position_3d_target[2], position_3d_source[2]]),
        )

        # Quaternions -> Euler
        quaternion_target = np.array(object_3d_target["object_data"]["cuboid"]["val"])[3:7]
        roll_target, pitch_target, yaw_target = Rotation.from_quat(quaternion_target).as_euler(
            "xyz", degrees=True
        )

        quaternion_source = np.array(object_3d_source["object_data"]["cuboid"]["val"])[3:7]
        roll_source, pitch_source, yaw_source = Rotation.from_quat(quaternion_source).as_euler(
            "xyz", degrees=True
        )

        if not same_origin:
            delta_yaw = 177.90826842 - 163.58774077
            yaw_source += delta_yaw

        # Average Euler angles
        roll_avg = (roll_target + roll_source) / 2.0
        pitch_avg = (pitch_target + pitch_source) / 2.0
        yaw_avg = (yaw_target + yaw_source) / 2.0

        # Back to quaternion
        quaternion_average = Rotation.from_euler("xyz", [roll_avg, pitch_avg, yaw_avg], degrees=True).as_quat()

        # Dimensions
        dimensions_3d_target = np.array(object_3d_target["object_data"]["cuboid"]["val"])[-3:]
        dimensions_3d_source = np.array(object_3d_source["object_data"]["cuboid"]["val"])[-3:]
        dimensions_3d_merged = (
            np.mean([dimensions_3d_target[0], dimensions_3d_source[0]]),
            np.mean([dimensions_3d_target[1], dimensions_3d_source[1]]),
            np.mean([dimensions_3d_target[2], dimensions_3d_source[2]]),
        )

        # --- Attributes from source ---
        if "attributes" in object_3d_source["object_data"]["cuboid"]:
            text_attrs_src = object_3d_source["object_data"]["cuboid"]["attributes"].get("text", [])
            num_attrs_src = object_3d_source["object_data"]["cuboid"]["attributes"].get("num", [])

            attr = get_attribute_by_name(text_attrs_src, "body_color")
            color_source = attr["val"] if attr is not None else ""

            attr = get_attribute_by_name(text_attrs_src, "occlusion_level")
            occlusion_level_source = attr["val"] if attr is not None else ""

            attr = get_attribute_by_name(num_attrs_src, "num_points")
            num_points_source = int(attr["val"]) if attr is not None else -1

            attr = get_attribute_by_name(num_attrs_src, "score")
            score_source = round(float(attr["val"]), 2) if attr is not None else -1

            attr = get_attribute_by_name(num_attrs_src, "number_of_trailers")
            number_of_trailers_source = int(attr["val"]) if attr is not None else 0
        else:
            color_source = ""
            occlusion_level_source = ""
            num_points_source = 0
            score_source = 0
            number_of_trailers_source = 0

        # --- Attributes from target ---
        if "attributes" in object_3d_target["object_data"]["cuboid"]:
            text_attrs_tgt = object_3d_target["object_data"]["cuboid"]["attributes"].get("text", [])
            num_attrs_tgt = object_3d_target["object_data"]["cuboid"]["attributes"].get("num", [])

            attr = get_attribute_by_name(text_attrs_tgt, "body_color")
            color_target = attr["val"] if attr is not None else ""

            attr = get_attribute_by_name(text_attrs_tgt, "occlusion_level")
            occlusion_level_target = attr["val"] if attr is not None else ""

            attr = get_attribute_by_name(num_attrs_tgt, "num_points")
            num_points_target = int(attr["val"]) if attr is not None else 0

            attr = get_attribute_by_name(num_attrs_tgt, "score")
            score_target = round(float(attr["val"]), 2) if attr is not None else -1

            attr = get_attribute_by_name(num_attrs_tgt, "number_of_trailers")
            number_of_trailers_target = int(attr["val"]) if attr is not None else 0
        else:
            color_target = ""
            occlusion_level_target = ""
            num_points_target = 0
            score_target = 0
            number_of_trailers_target = 0

        # --- Merge numeric attributes ---
        num_points_merged = (num_points_target + num_points_source) / 2.0
        score_merged = (score_target + score_source) / 2.0

        if score_target > score_source:
            color_merged = color_target
            number_of_trailers_merged = number_of_trailers_target
        else:
            color_merged = color_source
            number_of_trailers_merged = number_of_trailers_source

        # Occlusion merge
        if occlusion_level_target == occlusion_level_source:
            occlusion_level_merged = occlusion_level_target
        else:
            occl_t = occlusion_name_to_level_mapping.get(occlusion_level_target, 3)
            occl_s = occlusion_name_to_level_mapping.get(occlusion_level_source, 3)
            occlusion_level_merged = occlusion_level_to_name_mapping[min(occl_t, occl_s)]

        # Ensure "attributes" structure exists on target
        if "attributes" not in object_3d_target["object_data"]["cuboid"]:
            object_3d_target["object_data"]["cuboid"]["attributes"] = {}
        if "num" not in object_3d_target["object_data"]["cuboid"]["attributes"]:
            object_3d_target["object_data"]["cuboid"]["attributes"]["num"] = []
        if "text" not in object_3d_target["object_data"]["cuboid"]["attributes"]:
            object_3d_target["object_data"]["cuboid"]["attributes"]["text"] = []

        # Overwrite target object with merged data
        object_3d_target["object_data"]["cuboid"]["val"] = [
            position_3d_merged[0],
            position_3d_merged[1],
            position_3d_merged[2],
            quaternion_average[0],
            quaternion_average[1],
            quaternion_average[2],
            quaternion_average[3],
            dimensions_3d_merged[0],
            dimensions_3d_merged[1],
            dimensions_3d_merged[2],
        ]
        object_3d_target["object_data"]["cuboid"]["attributes"]["num"] = [
            {"name": "num_points", "val": num_points_merged},
            {"name": "score", "val": score_merged},
            {"name": "number_of_trailers", "val": number_of_trailers_merged},
        ]

        object_3d_target["object_data"]["cuboid"]["attributes"]["text"] = [
            {"name": "occlusion_level", "val": occlusion_level_merged},
            {"name": "sensor_id", "val": "s110_lidar_ouster_south_and_s110_lidar_ouster_north"},
        ]

    # --- Handle unmatched source objects ---
    objects_3d_source_ids_hex_unmatched = [objects_3d_source_ids_hex[x] for x in unmatched_source_ids]

    frame_obj_source = next(iter(json_label_target["openlabel"]["frames"].items()))[1]

    for id_hex, unmatch_id_source in zip(objects_3d_source_ids_hex_unmatched, unmatched_source_ids):
        object_3d_source = objects_3d_source_list[unmatch_id_source]

        position_3d_source = np.array(object_3d_source["object_data"]["cuboid"]["val"])[:3]

        if not same_origin:
            position_3d_source = np.matmul(
                transformation_matrix,
                np.array([position_3d_source[0], position_3d_source[1], position_3d_source[2], 1.0]),
            )

        object_3d_source["object_data"]["cuboid"]["val"][0] = position_3d_source[0]
        object_3d_source["object_data"]["cuboid"]["val"][1] = position_3d_source[1]
        object_3d_source["object_data"]["cuboid"]["val"][2] = position_3d_source[2]

        quaternion_source = np.array(object_3d_source["object_data"]["cuboid"]["val"])[3:7]
        roll_source, pitch_source, yaw_source = Rotation.from_quat(quaternion_source).as_euler(
            "xyz", degrees=True
        )

        if not same_origin:
            delta_yaw = 177.90826842 - 163.58774077
            yaw_source += delta_yaw

        quaternion_source_new = R.from_euler(
            seq="xyz",
            angles=[roll_source, pitch_source, yaw_source],
            degrees=True,
        ).as_quat()

        object_3d_source["object_data"]["cuboid"]["val"][3] = quaternion_source_new[0]
        object_3d_source["object_data"]["cuboid"]["val"][4] = quaternion_source_new[1]
        object_3d_source["object_data"]["cuboid"]["val"][5] = quaternion_source_new[2]
        object_3d_source["object_data"]["cuboid"]["val"][6] = quaternion_source_new[3]

        frame_obj_source["objects"][id_hex] = object_3d_source

    # --- Output file name, same logic as script ---
    if "s110_lidar_ouster_south" in boxes_file_name_target:
        output_file_name = boxes_file_name_target.replace(
            "s110_lidar_ouster_south",
            "s110_lidar_ouster_south_and_north_merged",
        )
    elif "s110_lidar_ouster_north" in boxes_file_name_target:
        output_file_name = boxes_file_name_target.replace(
            "s110_lidar_ouster_north",
            "s110_lidar_ouster_north_and_south_merged",
        )
    elif "s110_camera_basler_south1_8mm" in boxes_file_name_target:
        output_file_name = boxes_file_name_target.replace(
            "s110_camera_basler_south1_8mm",
            "s110_camera_basler_south1_8mm_and_south2_8mm_merged",
        )
    else:
        # fallback
        output_file_name = boxes_file_name_target.replace(".json", "_merged.json")

    # Update frame_properties -> multiple point cloud filenames
    if (
        "frame_properties" in frame_obj_source
        and "point_cloud_file_name" in frame_obj_source["frame_properties"]
    ):
        del frame_obj_source["frame_properties"]["point_cloud_file_name"]
        frame_obj_source["frame_properties"]["point_cloud_file_names"] = [
            boxes_file_name_target.replace(".json", ".pcd"),
            boxes_file_name_source.replace(".json", ".pcd"),
        ]

    # Store fused JSON (output_folder_path_fused_boxes is Path)
    store_boxes_in_open_label(json_label_target, output_folder_path_fused_boxes, output_file_name)

    # Print per-frame info so you can verify mapping
    print(f"[{frame_id:04d}] target: {boxes_file_name_target} | "
          f"source: {boxes_file_name_source} -> fused: {output_file_name}")

    frame_id += 1

print("Total matched object pairs:", num_matches)
print("Fusion done. Fused labels saved to:", output_folder_path_fused_boxes)


Fusing frames:   1%|▍                                                                | 15/2160 [00:00<00:29, 71.88it/s]

[WARN] Frame 0: more source objects than target objects.
[0000] target: 1646667310_042939725_s110_lidar_ouster_north.json | source: 1646667310_053239541_s110_lidar_ouster_south.json -> fused: 1646667310_042939725_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 1: more source objects than target objects.
[0001] target: 1646667310_143300724_s110_lidar_ouster_north.json | source: 1646667310_150192886_s110_lidar_ouster_south.json -> fused: 1646667310_143300724_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 2: more source objects than target objects.
[0002] target: 1646667310_238868597_s110_lidar_ouster_north.json | source: 1646667310_256197306_s110_lidar_ouster_south.json -> fused: 1646667310_238868597_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 3: more source objects than target objects.
[0003] target: 1646667310_338451604_s110_lidar_ouster_north.json | source: 1646667310_352129938_s110_lidar_ouster_south.json -> fused: 1646667310_338451604_s110_lidar

Fusing frames:   1%|▉                                                                | 31/2160 [00:00<00:28, 74.53it/s]

[WARN] Frame 15: more source objects than target objects.
[0015] target: 1646667311_947409606_s110_lidar_ouster_north.json | source: 1646667311_952944257_s110_lidar_ouster_south.json -> fused: 1646667311_947409606_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 16: more source objects than target objects.
[0016] target: 1646667312_044607640_s110_lidar_ouster_north.json | source: 1646667312_052276201_s110_lidar_ouster_south.json -> fused: 1646667312_044607640_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 17: more source objects than target objects.
[0017] target: 1646667312_144451675_s110_lidar_ouster_north.json | source: 1646667312_149771453_s110_lidar_ouster_south.json -> fused: 1646667312_144451675_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 18: more source objects than target objects.
[0018] target: 1646667312_243894322_s110_lidar_ouster_north.json | source: 1646667312_257338745_s110_lidar_ouster_south.json -> fused: 1646667312_243894322_s110_l

Fusing frames:   2%|█▎                                                               | 45/2160 [00:00<00:40, 52.27it/s]

...

[WARN] Frame 2111: more source objects than target objects.
[2111] target: 1653330113_998758606_s110_lidar_ouster_north.json | source: 1653330114_020489187_s110_lidar_ouster_south.json -> fused: 1653330113_998758606_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 2112: more source objects than target objects.
[2112] target: 1653330114_096325065_s110_lidar_ouster_north.json | source: 1653330114_114170389_s110_lidar_ouster_south.json -> fused: 1653330114_096325065_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 2113: more source objects than target objects.
[2113] target: 1653330114_291432796_s110_lidar_ouster_north.json | source: 1653330114_209585247_s110_lidar_ouster_south.json -> fused: 1653330114_291432796_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 2114: more source objects than target objects.
[2114] target: 1653330114_291432797_s110_lidar_ouster_north.json | source: 1653330114_306220105_s110_lidar_ouster_south.json -> fused: 1653330114_29143279

Fusing frames:  99%|█████████████████████████████████████████████████████████████▋| 2147/2160 [00:41<00:00, 125.46it/s]

[2142] target: 1653330117_198618417_s110_lidar_ouster_north.json | source: 1653330117_214806140_s110_lidar_ouster_south.json -> fused: 1653330117_198618417_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 2143: more source objects than target objects.
[2143] target: 1653330117_303233226_s110_lidar_ouster_north.json | source: 1653330117_310367466_s110_lidar_ouster_south.json -> fused: 1653330117_303233226_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 2144: more source objects than target objects.
[2144] target: 1653330117_396457067_s110_lidar_ouster_north.json | source: 1653330117_412134698_s110_lidar_ouster_south.json -> fused: 1653330117_396457067_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 2145: more source objects than target objects.
[2145] target: 1653330117_603658810_s110_lidar_ouster_north.json | source: 1653330117_610545044_s110_lidar_ouster_south.json -> fused: 1653330117_603658810_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame

Fusing frames: 100%|███████████████████████████████████████████████████████████████| 2160/2160 [00:42<00:00, 51.21it/s]

[WARN] Frame 2158: more source objects than target objects.
[2158] target: 1653330118_892246465_s110_lidar_ouster_north.json | source: 1653330118_910670367_s110_lidar_ouster_south.json -> fused: 1653330118_892246465_s110_lidar_ouster_north_and_south_merged.json
[WARN] Frame 2159: more source objects than target objects.
[2159] target: 1653330118_996008667_s110_lidar_ouster_north.json | source: 1653330119_016082157_s110_lidar_ouster_south.json -> fused: 1653330118_996008667_s110_lidar_ouster_north_and_south_merged.json
Total matched object pairs: 1960
Fusion done. Fused labels saved to: C:\Users\mcsmu\Desktop\mm3\Fused json
